<a href="https://colab.research.google.com/github/RubensBenevides/Vizualizando-e-operando-com-nuvens-de-pontos-em-Caderno-Jupyter/blob/master/Pre_processamento_registro_e_segmentacao_de_nuvens_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
**# Vizualizando e operando com nuvens de pontos em Caderno Jupyter**
---
---

Preparar o ambiente no Google Drive:

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Instalar e importar bibliotecas para uso geral e manipulação 3D 

In [ ]:
# Uso geral
!pip install numpy
!pip install pandas
import time
!ls                  # para importar dados do Google Drive

# Manipulação e vizualização de nuvens de pontos
!pip install ipyvolume 
!pip install pyntcloud
!pip install open3d

In [3]:
# importar
import numpy as np
import pandas as pd
import ipyvolume as ipv
import open3d as o3d
from pyntcloud import PyntCloud as pytc
import copy
import time

## **1 - Importar nuvem de pontos 3D**
Uma nuvem de pontos ter diversos formatos, o mais básico deles é simplesmente um arquivo .txt (ascII) de 3 colunas contendo xyz. Quatro das bibliotecas utilizadas permitem esse tipo de importação, mas algumas são mais eficientes que outras. Outra coisa importante é a formatação e precisão do dado, que pode ser de simples precisão (float32) ou dupla precisão (flaot64). Faremos alguns testes com a importação de uma nuvem média (269 mil pontos) que tem colunas separadas por vírgulas. Utilizaremos todas as bibliotecas que permitem isso:

### **1.1 Import pela Numpy**
A numpy lê a nuvem como uma matriz de n linhas e 3 colunas, embora seja uma biblioteca extremamente otimizada e rápida na manipulação de arrays, é um pouco lenta na importação.

In [ ]:
tempo_i = time.time()
pc_numpy  = np.loadtxt('/content/gdrive/MyDrive/Mestrado/Proc_de_nuvens_de_pts_3d/nuvens_3D_variadas/s0.txt')
tempo_f = time.time() - tempo_i

print(pc_numpy)
print(len(pc_numpy))
print("Tempo para ler a nuvem: %.3f" %tempo_f)

[[-40.44937515  22.50149536  -3.76761794]
 [-40.52937317  22.64649582  -3.77761793]
 [-40.38437653  22.6914959   -3.76761794]
 ...
 [-18.40437508 -10.08850479   7.09238195]
 [-18.38937378 -10.0035038    7.07238197]
 [-18.23937416  -7.22850513   7.07238197]]
18421
Tempo para ler a nuvem: 0.154


### **1.2 Import pela Pandas**
A Pandas lê a nuvem como um dataframe e nomeia as colunas dos dados com a primeira linha, para que isto não aconteça damos nomes às colunas: 'x', 'y' e 'z'. A leitura é 6x mais rápida do que pela numpy.

In [ ]:
tempo_i = time.time()
pc_pandas = pd.read_csv('/content/gdrive/MyDrive/Mestrado/Proc_de_nuvens_de_pts_3d/nuvens_3D_variadas/s1.txt', names=["x","y","z"])
tempo_f = time.time() - tempo_i

print(pc_pandas)
type(pc_pandas)
print("Tempo para ler a nuvem: %.3f" %tempo_f)

                                          x   y   z
0      -24.82820892 21.61056519 -4.49321604 NaN NaN
1      -24.66820908 21.59556580 -4.47821617 NaN NaN
2      -24.59571075 21.66806602 -4.47571611 NaN NaN
3      -24.49821091 21.72556686 -4.46821594 NaN NaN
4      -24.31320953 21.77056503 -4.45821619 NaN NaN
...                                     ...  ..  ..
19046  -2.64821100 -37.38943481 13.30678368 NaN NaN
19047  -2.54821110 -37.39943314 13.30678368 NaN NaN
19048  -3.63821101 -37.29443359 13.30678368 NaN NaN
19049  -3.53821111 -37.31443405 13.31178474 NaN NaN
19050  -3.43821001 -37.32443619 13.31178474 NaN NaN

[19051 rows x 3 columns]
Tempo para ler a nuvem: 0.023


###**1.3 Import pela Pyntcloud**
Por trás da Pyntcloud roda a função read_csv da pandas, mas a pyntcloud cria um objeto próprio, não um dataframe. Por ser um biblioteca específica para nuvens de pontos consegue ler a nuvem no formato .pcd, que é padrão neste tipo de dado. O tempo de leitura é igual a numpy

In [ ]:
tempo_i = time.time()
pc_pyntcloud = pytc.from_file('/content/gdrive/MyDrive/Mestrado/Proc_de_nuvens_de_pts_3d/nuvens_3D_variadas/s0.pcd')
tempo_f = time.time() - tempo_i

print(pc_pyntcloud)
type(pc_pyntcloud)
print("Tempo para ler a nuvem: %.3f" %tempo_f)

PyntCloud
18421 points with 0 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: 9.542437737763976e-07, -6.361625537465443e-07, 7.952031637614709e-07
Other attributes:

Tempo para ler a nuvem: 0.159


###**1.4 Import pela Open3D**
A Open3D consegue ler arquivos pcd, ply e ascii como: .xyz, .xyzrgb, .xyzn e .pts. Todavia, no caso dos tipos ascii, ela não consegue ler dados separados por vírgula. Importaremos a nuvem em formato .pcd que é mais rápido.

In [ ]:
tempo_i = time.time()
pc_open3d = o3d.io.read_point_cloud('/content/gdrive/MyDrive/Mestrado/Proc_de_nuvens_de_pts_3d/nuvens_3D_variadas/s0.pcd')
tempo_f = time.time() - tempo_i

print(pc_open3d)
type(pc_open3d)
print("Tempo para ler a nuvem: %.3f" %tempo_f)

PointCloud with 18421 points.
Tempo para ler a nuvem: 0.022


##**2 - Visualização**

###**2.1 Visualização com a PyntCloud**
A visualização da pyntcloud aparece com os pontos em tamanhos exagerados, mas é possível alterar isso facilmente para que a nuvem apareça normalmente

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
pc_pyntcloud.plot()

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(9.542437737763976e-07, 54.166495640692915, 1…

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

### **2.2 Visualização com a ipyVolume**
A ipyVolume permite animações e muitos outros recursos da ipywidgets incorporados na sua implementação.
Todavia, a renderização é mais complexa e pode ficar lenta dependendo do seu hardware. Para plotar os pontos precisamos das colunas da matriz numpy.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
x = pc_numpy[:,0];y = pc_numpy[:,1];z = pc_numpy[:,2]
ipv.quickscatter(x, y, z, size=0.5, marker="sphere")

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

### **2.3 Visualização com a Open3d** 
A Open3d não suporta visualizações dentro de cadernos jupyter, e a nuvem não aparece se os dados estiverem em coordenadas UTM. Para contornar isso basta centralizar a nuvem. Além disso, o objeto tem que ser passado como uma lista para a função draw_geometries

In [ ]:
o3d.visualization.draw_geometries([pc_open3d])

### **2.4 Visualização com a Plotly**
A unica que funciona no colab mesmo é a Plotly

In [ ]:
# install e import bbox
!pip install -q bbox-utils
from bbox_utils.point_cloud import PointCloud
import plotly.graph_objects as go

In [ ]:
pcd = PointCloud.load_from_file('/content/gdrive/MyDrive/Mestrado/Proc_de_nuvens_de_pts_3d/nuvens_3D_variadas/s0.pcd')
_ = pcd.display()

Essa funcao substitui a função de visualizacao da Open3D pela da plotly

In [ ]:
def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()
        
        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((1.0, 0.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            triangles = np.asarray(geometry.triangles)
            vertices = np.asarray(geometry.vertices)
            colors = None
            if geometry.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)
            
            mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
            graph_objects.append(mesh_3d)
        
    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

o3d.visualization.draw_geometries = draw_geometries # altera a funcao da open3D para usar a da plotly

## **3 - Kd-trees & Octrees**
Kd-trees e octrees são estruturas utilizadas quando manipulamos nuvens de pontos. A maioria dos filtros e diversas outras funcionalidades necessitam de kd-trees e octrees para funcionar. O nome tree (árvore) se deve à forma como o objeto é criado através de subdivisões do espaço. Apenas as bibliotecas Open3D e Pyncloud, específicas para nuvens de pontos, trabalham com este objeto.

### **3.1 Kd-tree (Pyntcloud)**
No exemplo abaixo utilizamos uma kd-tree gerada pela pyntcloud para selecionar os k vizinhos mais próximos de cada ponto da nuvem. A estrutura gerada é uma matriz com k colunas dos índices dos pts mais próximos.

In [ ]:
# Adicionar estrutura kdtree na nuvem:
kdtree_pyntcloud = pc_pyntcloud.add_structure("kdtree")

# Guardar os índices dos 6 vizinhos mais próximos de cada ponto:
k_vizinhos = pc_pyntcloud.get_neighbors(k=6,kdtree=kdtree_pyntcloud)

print(k_vizinhos)

###**3.2 Voxels (Pyntcloud)**

A função abaixo permite subdividir o espaço ocupado pela nuvem com uma octree. Os parâmetros da subdivisão podem ser a quantidade de voxels finais desejados em cada eixo (n_x=1, n_y=1, n_z=1) ou o tamanho dos voxels criados em cada eixo (size_x=1.0, size_y=1.0, size_z=1.0). Colocar valores pequenos nos tamanhos (abaixo de 0.5) ou muito grandes nas quantidades finais (acima de 100) costuma travar o Jupyter por falta de memória. Setar 'None' nos tamanhos se quiser usar as quantidades.

In [ ]:
# Adicionar octree na nuvem importada com a Pyntcloud
voxelgrid_pyntcloud = pc_pyntcloud.add_structure("voxelgrid", n_x=1, n_y=1, n_z=1, size_x=1.0, size_y=1.0, size_z=1.0)
print(voxelgrid_pyntcloud)

### **3.3 Kd-tree (Open3D)**
Kdtrees na Open3D são implementadas utilizando o algoritmo FLANN (Fast Library for Approximate Nearest Neighbors)

In [ ]:
# Pinta a nuvem de amarelo
pc_open3d.paint_uniform_color([0.8, 0.8, 0])

# Cria a kd-tree na nuvem
pcd_tree = o3d.geometry.KDTreeFlann(pc_open3d)

# Pinta o ponto de indice 1000 de vermelho:
pc_open3d.colors[1000] = [1, 0, 0]

# Procura os 1000 pontos vizinhos do ponto anterior
[k, idx, _] = pcd_tree.search_knn_vector_3d(pc_open3d.points[1000],1000)

# Pinta os vizinhos de azul:
np.asarray(pc_open3d.colors)[idx[1:], :] = [0, 0, 1]

# Encontra os vizinhos com distância menor que 5 m:
[k, idx, _] = pcd_tree.search_radius_vector_3d(pc_open3d.points[1000],5.0)

# Pinta os vizinhos dentro dos 5 m de preto:
np.asarray(pc_open3d.colors)[idx[1:], :] = [0, 0, 0]

# Plota a nuvem:
o3d.visualization.draw_geometries([pc_open3d])

### **3.4 Voxels (Open3D)**
A linha baixo cria uma octree com 64 voxels utilizando a Open3D, para isso ela divide um cubo de 1 m³ centrado em [0,0,0] em cubos menores de 0.25 m de largura, o que dá 4x4x4 voxels.

In [ ]:
octree = o3d.geometry.VoxelGrid.create_dense([0.0, 0.0 ,0.0], [0.3, 0.0, 0.7], 0.25, 1.0, 1.0, 1.0)
o3d.visualization.draw_geometries([octree])

## **4 - Pre-Processamento de nuvens**
Quando temos uma nuvem muito grande é coveniente que se faça uma voxialização para reduzir seu tamanho. Além de deixá-la com densidade uniforme, essa operação consegue reduzir a quantidade de pontos preservando a forma dos objetos. Para o registro e a segmentação a voxialização é crítica, pois a operação demanda tempo exponencialmente maior de acordo com o tamanho da nuvem.


### **4.1 Voxelização (Pyntcloud)**

In [ ]:
pc_voxializada_pyntcloud = pc_pyntcloud.get_sample(name='voxelgrid_centers',as_PyntCloud=True,voxelgrid_id=voxelgrid_pyntcloud)
print(pc_voxializada_pyntcloud)
# Desenhar nuvem voxializada:
pc_voxializada_pyntcloud.plot()

### **4.2 Voxializacao (Open3d)**

In [ ]:
pc_voxializada_open3d = pc_open3d.voxel_down_sample(voxel_size=0.1)
o3d.visualization.draw_geometries([pc_voxializada_open3d])

### **4.3 Filtragem (Pyntcoud)**
Filtragens são feitas considerando uma vizinhança ao redor de um ponto. Essa vizinhança pode ser definida como um raio geométrico ao redor do ponto ou por um desvio-padrão. No primeiro caso temos o filtro por raio, no segundo o filtro estatístico. Recomenda-se fazer a filtragem sempre após a voxialização. A filtragem utiliza kd-trees para encontrar vizinhos mais próximos, na pyntcloud precisamos construir a kd-tree da nuvem que será filtrada:

In [ ]:
# RADIOS OUTLIER REMOVAL (ROR)
# Entradas: raio, knn_vizinhos, kdtree;   
# Saídas: índices dos pontos filtrados.

kdtree_filtragem = pc_voxializada_pyntcloud.add_structure("kdtree")
idx = pc_pyntcloud.get_filter(name='ROR', k=10, r=1.0, kdtree_id = kdtree_filtragem)

pts_filtrados = len(idx)-sum(idx)
print("pontos filtrados: %d" %pts_filtrados)

In [ ]:
# STATISTIICAL OUTLIER REMOVAL (SOR)
# Entradas: desvio_padrao, knn_vizinhos, kdtree;   
# Saídas: índices dos pontos filtrados.
idx = pc_voxializada_pyntcloud.get_filter(name='SOR', k=6,z_max=1.2, kdtree_id = kdtree_filtragem)

pts_filtrados = len(idx)-sum(idx)
print("pontos filtrados: %d" %pts_filtrados)

### **4.3 Filtragem (Open3D)**
Os mesmos filtros aplicados na mesma nuvem com os mesmos parâmetros produzem resultados diferentes na Open3D. Isso ocorre porque o algoritmo da kd-tree é diferente e as voxializações produziram nuvens com quantidades de pts. diferentes.

In [ ]:
# FILTRO POR RAIO.
# ENTRADAS: Raio e Quantidade mínima de pontos.
# SAÍDAS: nuvem filtrada e índices desses pontos.
pc_filtrada_raio, ind = pc_voxializada_open3d.remove_radius_outlier(nb_points=6, radius=1.5)
print(pc_filtrada_raio)
o3d.visualization.draw_geometries([pc_filtrada_raio])

pts_filtrados = len(pc_voxializada_open3d.points)-len(pc_filtrada_raio.points)
print("pontos filtrados: %d" %pts_filtrados)

In [ ]:
# FILTRO ESTATÍTISTICO.
# ENTRADAS: Desvio-Padrão e Quantidade mínima de pontos.
# SAÍDAS: nuvem filtrada e índices desses pontos.
pc_filtrada_estat, ind = pc_voxializada_open3d.remove_statistical_outlier(nb_neighbors=6, std_ratio=1.2)
print(pc_filtrada_estat)
o3d.visualization.draw_geometries([pc_filtrada_estat])

pts_filtrados = len(pc_voxializada_open3d.points)-len(pc_filtrada_estat.points)
print("pontos filtrados: %d" %pts_filtrados)

## **5 - Registro de pares de nuvens de pontos**
Apenas a Open3D tem funções implementadas para fazer o registro de nuvens de pontos. Essas funções se subdividem em registros globais para alinhamento inicial e registros finos, que consistem em variações do ICP (Iterative Closest Point) para melhorar o resultado do registro global. O registro é simplesmente o alinhamento de duas nuvens no mesmo referencial, portanto, a sua saída é uma matriz de transformação homogênia que inclui uma rotação e uma translação em uma única matriz 4x4.

Para efetuar o registro precisamos de duas nuvens, das normais dos pontos e dos descritores dos pontos. E para o cálculo das normais e dos descritores precisamos da kd-tree. A seguir fazemos todo esse pre-processamento:

In [ ]:
# Importar duas nuvens com sobreposição e posições translocadas/rotacionadas:
pc_open3d_1 = o3d.io.read_point_cloud('/content/gdrive/MyDrive/Mestrado/Proc_de_nuvens_de_pts_3d/nuvens_3D_variadas/s0.pcd')
pc_open3d_2 = o3d.io.read_point_cloud('/content/gdrive/MyDrive/Mestrado/Proc_de_nuvens_de_pts_3d/nuvens_3D_variadas/s1.pcd')

# Voxializar as nuvens:
pc_voxializada_1 = pc_open3d_1.voxel_down_sample(voxel_size=0.2)
pc_voxializada_2 = pc_open3d_2.voxel_down_sample(voxel_size=0.2)

# Filtrar as nuvens (filtro estatístico):
pc_filtrada_1, ind = pc_voxializada_1.remove_statistical_outlier(nb_neighbors=6, std_ratio=2.0)
pc_filtrada_2, ind = pc_voxializada_2.remove_statistical_outlier(nb_neighbors=6, std_ratio=2.0)

# Construir kd-tree para calcular normais (20 cm) e outra para calcular descritores (2 m):
kd_tree_normais     = o3d.geometry.KDTreeSearchParamHybrid(radius=0.2, max_nn=10)
kd_tree_descritores = o3d.geometry.KDTreeSearchParamHybrid(radius=2.0, max_nn=100)

# Calcular normais das nuvens:
normais_1 = pc_filtrada_1.estimate_normals(kd_tree_normais)
normais_2 = pc_filtrada_2.estimate_normals(kd_tree_normais)

# Calcular descritores das nuvens:
pc_fpfh_1 = o3d.pipelines.registration.compute_fpfh_feature(pc_filtrada_1,kd_tree_descritores)
pc_fpfh_2 = o3d.pipelines.registration.compute_fpfh_feature(pc_filtrada_2,kd_tree_descritores)

print(pc_filtrada_1)
print(pc_filtrada_2)
pc_filtrada_1.paint_uniform_color([0.8, 0.8, 0])
pc_filtrada_2.paint_uniform_color([0, 0, 0.9])
o3d.visualization.draw_geometries([pc_filtrada_1,pc_filtrada_2])

In [ ]:
# Função para desenhar o resultado do registro com duas cores diferentes para cada nuvem
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([0.8, 0.8, 0])
    target_temp.paint_uniform_color([0, 0, 0.9])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [ ]:
def registro_RANSAC_based(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
                          distance_threshold = voxel_size * 1.5
                          result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
                                                                                source_down, # nuvem 1
                                                                                target_down, # nuvem 2
                                                                                source_fpfh, # features 1
                                                                                target_fpfh, # features 2
                                                                                True, # Se o registro deve filtrar corres
                                                                                distance_threshold,
                                                                                o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
                                                                                3,
                                                                                [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                                                                                 o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
                                                                                 o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
                          return result

### **5.1 Registro Global 1: RANSAC com base no match de correspondências**

In [ ]:
resultado_RANSAC = registro_RANSAC_based(pc_filtrada_1,
                                         pc_filtrada_2,
                                         pc_fpfh_1,
                                         pc_fpfh_2,
                                         voxel_size=1.0)

print(resultado_RANSAC)
draw_registration_result(pc_filtrada_1, pc_filtrada_2, resultado_RANSAC.transformation)

#### **5.1.1 Refinamento do registro RANSAC por ICP ponto-a-ponto**

In [ ]:
limite_distancia = 0.05 # 5 cm = 1/3 do voxel de amostragem
tipo_registro = o3d.pipelines.registration.TransformationEstimationPointToPoint()
resultado_icp = o3d.pipelines.registration.registration_icp(pc_filtrada_1,
                                                            pc_filtrada_2, 
                                                            limite_distancia,
                                                            resultado_RANSAC.transformation,
                                                            tipo_registro)
print(resultado_icp)
draw_registration_result(pc_filtrada_1, pc_filtrada_2, resultado_icp.transformation)
resultado_icp.transformation

#### **5.1.2 Refinamento do registro RANSAC por ICP ponto-a-plano**

In [ ]:
limite_distancia = 0.05 # 5 cm = 1/3 do voxel de amostragem
tipo_registro = o3d.pipelines.registration.TransformationEstimationPointToPlane()
resultado_icp = o3d.pipelines.registration.registration_icp(pc_filtrada_1,
                                                            pc_filtrada_2, 
                                                            limite_distancia,
                                                            resultado_RANSAC.transformation,
                                                            tipo_registro)
print(resultado_icp)
draw_registration_result(pc_filtrada_1, pc_filtrada_2, resultado_icp.transformation)
resultado_icp.transformation

### **5.2 Registro Global 2: Fast Global Registration (FGR) (ZHOU, 2016)**

In [ ]:
# Definir parâmetros do registro FGR
registro_FGR = o3d.pipelines.registration.FastGlobalRegistrationOption(division_factor = 1.4, # padrão: 1.4 
                                                             use_absolute_scale = True,       # padrão: False
                                                             decrease_mu = True,              # padrão: False
                                                             maximum_correspondence_distance = 0.015,              
                                                             iteration_number = 640,          # padrão: 64 
                                                             tuple_scale = 0.95,              # padrão: 0.95
                                                             maximum_tuple_count = 1000)      # padrão: 300
# Executar registro FGR
resultado_FGR = o3d.pipelines.registration.registration_fast_based_on_feature_matching(pc_filtrada_1,
                                                                             pc_filtrada_2, 
                                                                             pc_fpfh_1,
                                                                             pc_fpfh_2,
                                                                             registro_FGR)

draw_registration_result(pc_filtrada_1, pc_filtrada_2, resultado_FGR.transformation)
print(resultado_FGR)
resultado_FGR.transformation

#### **5.2.1 Refinamento do FGR por ICP-ponto-a-ponto**

In [ ]:
limite_distancia = 0.05 # 5 cm = 1/3 do voxel de amostragem
tipo_registro = o3d.pipelines.registration.TransformationEstimationPointToPoint()
resultado_icp = o3d.pipelines.registration.registration_icp(pc_filtrada_1,
                                                            pc_filtrada_2, 
                                                            limite_distancia,
                                                            resultado_FGR.transformation,
                                                            tipo_registro)
print(resultado_icp)
draw_registration_result(pc_filtrada_1, pc_filtrada_2, resultado_icp.transformation)
resultado_icp.transformation

RegistrationResult with fitness=5.358747e-02, inlier_rmse=3.678366e-02, and correspondence_set size of 537
Access transformation to get result.


array([[ 0.99748226,  0.06655329, -0.02449085, -1.16196272],
       [-0.06642688,  0.99777361,  0.0059402 ,  0.0573015 ],
       [ 0.02483166, -0.0042984 ,  0.99968241, -0.0030093 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

#### **5.2.2 Refinamento do FGR 2 por ICP-ponto-a-plano**

In [ ]:
0.05 # 5 cm = 1/3 do voxel de amostragem
tipo_registro = o3d.pipelines.registration.TransformationEstimationPointToPlane()
resultado_icp = o3d.pipelines.registration.registration_icp(pc_filtrada_1,
                                                  pc_filtrada_2, 
                                                  limite_distancia,
                                                  resultado_FGR.transformation,
                                                  tipo_registro)
print(resultado_icp)
draw_registration_result(pc_filtrada_1, pc_filtrada_2, resultado_icp.transformation)
resultado_icp.transformation

RegistrationResult with fitness=5.039417e-02, inlier_rmse=3.758362e-02, and correspondence_set size of 505
Access transformation to get result.


array([[ 0.99757741,  0.06534713, -0.02385514, -1.16774779],
       [-0.06523806,  0.99785553,  0.00532295,  0.05877481],
       [ 0.02415182, -0.00375379,  0.99970125, -0.01088358],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

## **6 - Segmentação**
A segmentação de nuvens de pontos consiste em separar estruturas com geometrias iguais. O exemplo mais básico que podemos dar é separar estruturas planas de não planas. Faremos isso decompondo a MVC da vizinhança de um ponto em autovalores, com eles calculam-se índices de planaridade, esfericidade, etc, que descrevem a vizinha do ponto. Utilizaremos a pyntcloud porque nela estes cálculos estão implementados.

### **6.1 Segmentação por Machine Learning (k-medias) com atributos dos pontos vizinhos**  

In [ ]:
# Adicionar estrutura kdtree na nuvem
pc_kdtree  = pc_pyntcloud.add_structure("kdtree")

# Vizinhanças de cada ponto k
k_vizinhos = pc_pyntcloud.get_neighbors(k=20,kdtree=pc_kdtree)

# Adicionar colunas dos autovalores (scalar fields)
autovalores = pc_pyntcloud.add_scalar_field("eigen_values", k_neighbors=k_vizinhos)

# Adicionar Colunas das medidas dimensionais
# Planaridade: (e2-e3/e1)
eigen = pc_pyntcloud.add_scalar_field("planarity",ev=autovalores)
# Anisotropia: (e1-e3/e1)
eigen = pc_pyntcloud.add_scalar_field("anisotropy",ev=autovalores)
# Esfericidade: (e3/e1)
eigen = pc_pyntcloud.add_scalar_field("sphericity",ev=autovalores)
# Variância: (e1*e2*e3)^1/2
eigen = pc_pyntcloud.add_scalar_field("omnivariance",ev=autovalores)
# Entropia: sum[ei*ln(ei)]
eigen = pc_pyntcloud.add_scalar_field("eigenentropy",ev=autovalores)

pc_pyntcloud.points.head()

/usr/local/lib/python3.7/dist-packages/pyntcloud/scalar_fields/eigenvalues.py:50: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.7/dist-packages/pyntcloud/scalar_fields/eigenvalues.py:50: RuntimeWarning:

invalid value encountered in multiply



,x,y,z,e1(21),e2(21),e3(21),planarity(21),anisotropy(21),sphericity(21),omnivariance(21),eigenentropy(21)
0,-40.449375,22.501495,-3.767618,0.423438,0.011726,0.000002,0.027687,0.999995,0.000005,0.002227,0.416042
1,-40.529373,22.646496,-3.777618,0.423438,0.011726,0.000002,0.027687,0.999995,0.000005,0.002227,0.416042
2,-40.384377,22.691496,-3.767618,0.423438,0.011726,0.000002,0.027687,0.999995,0.000005,0.002227,0.416042
3,-40.319374,22.776495,-3.767618,0.423438,0.011726,0.000002,0.027687,0.999995,0.000005,0.002227,0.416042
4,-40.279373,22.871496,-3.767618,0.423438,0.011726,0.000002,0.027687,0.999995,0.000005,0.002227,0.416042


In [ ]:
# Retira do dataFrame todos os pontos que não passam no critério
planos = pc_pyntcloud.points[pc_pyntcloud.points["planarity(21)"] > 0.8]
arvores = pc_pyntcloud.points[pc_pyntcloud.points["planarity(21)"] < 0.2]

In [ ]:
# Converte para apenas as primeiras 3 colunas e salva a nuvem de objetos plano
a = planos.values
a = a[:,0:3]
pts = pd.DataFrame(a)
pts.rename(columns={0:'x',1:'y',2:'x'},inplace=True)
#pts.to_csv('/home/rubens/Nuvem_Pontos_3D/nuvem_planos.csv')


In [ ]:
# Converte para apenas as primeiras 3 colunas e salva a nuvem de objetos não-planos
a = arvores.values
a = a[:,0:3]
pts = pd.DataFrame(a)
pts.rename(columns={0:'x',1:'y',2:'x'},inplace=True)
# pts.to_csv('/home/rubens/Nuvem_Pontos_3D/nuvem_arvores.csv')
# A função de escrita da biblioteca PyntCloud não funciona

In [ ]:
# Nuvem dos objetos planos (pontos com planaridade superior que 0.80)
nuvem_planos = pytc.from_file("/home/rubens/Nuvem_Pontos_3D/nuvem_planos.csv",sep=",",header=0,names=["x","y","z"])
nuvem_planos.plot()

In [ ]:
# Nuvem dos objetos não planos (pontos com planaridade inferior a 0.2)
nuvem_arvores = pytc.from_file("/home/rubens/Nuvem_Pontos_3D/nuvem_arvores.csv",sep=",",header=0,names=["x","y","z"])
nuvem_arvores.plot()